In [ ]:
pip install scikit-learn statsmodels -qq

In [ ]:
import pandas as pd
import requests
import time
import os
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
client_id = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
client_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXX'
token_url = "https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
base_url = "https://tdx.transportdata.tw/api/advanced/v2/Bike"
availability_endpoint = "/Availability/NearBy"
station_endpoint = "/Station/NearBy"
top = "30"
lat = "22.62752590909029"
lng = "120.26465291318681"
radius = "1000"
AVAILABILITY_URL = f"{base_url}{availability_endpoint}?%24top={top}&%24spatialFilter=nearby%28{lat}%2C%20{lng}%2C%20{radius}%29&%24format=JSON"
STATION_URL = f"{base_url}{station_endpoint}?%24top={top}&%24spatialFilter=nearby%28{lat}%2C%20{lng}%2C%20{radius}%29&%24format=JSON"

In [ ]:
class TDX():
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret

    def get_token(self):
        headers = {'content-type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        response = requests.post(token_url, headers=headers, data=data)
        return response.json()['access_token']

    def get_response(self, url):
        headers = {'authorization': f'Bearer {self.get_token()}'}
        response = requests.get(url, headers=headers)
        return response.json()

In [ ]:
def collect_data():
    try:
        tdx = TDX(client_id, client_secret)
        availability_response = tdx.get_response(AVAILABILITY_URL)
        station_response = tdx.get_response(STATION_URL)
        df_result = pd.DataFrame()
        for station in station_response:
            station_id = station['StationID']
            station_name = station['StationName']['Zh_tw']
            station_address = station['StationAddress']['Zh_tw']
            bikes_capacity = station['BikesCapacity']
            for avail in availability_response:
                if avail['StationID'] == station_id:
                    available_rent_bikes = avail['AvailableRentBikes']
                    available_rent_electricbikes = avail['AvailableRentBikesDetail']['ElectricBikes']
                    available_rent_generalbikes = avail['AvailableRentBikesDetail']['GeneralBikes']
                    available_return_bikes = avail['AvailableReturnBikes']
                    update_time = avail['UpdateTime']
                    break
            df_result = df_result.append({
                'StationID': station_id,
                'StationName': station_name,
                'StationAddress': station_address,
                'BikesCapacity': bikes_capacity,
                'AvailableRentBikes': available_rent_bikes,
                'ElectricBikes': available_rent_electricbikes,
                'GeneralBikes': available_rent_generalbikes,
                'AvailableReturnBikes': available_return_bikes,
                'UpdateTime': update_time
            }, ignore_index=True)
        csv_path = '國立中山大學幾何中心周圍1公里Youbike站點即時狀態.csv'
        if os.path.isfile(csv_path):
            df_result.to_csv(csv_path, mode='a', header=False, index=False)
        else:
            df_result.to_csv(csv_path, mode='w', index=False)
    except Exception as e:
        print(f"Error occurred: {e}")

In [ ]:
def predict_zero_bikes_or_slots_with_arima(data):
    stations_with_zero_rent = []
    stations_with_zero_return = []

    unique_stations = data['StationID'].unique()

    for station in unique_stations:
        station_data = data[data['StationID'] == station].reset_index(drop=True)


        if len(station_data) < 5:
            print(f"Skipping station {station} due to insufficient data.")
            continue
        model_rent = ARIMA(station_data['AvailableRentBikes'], order=(1,1,0))
        model_rent_fit = model_rent.fit()
        forecast_rent = model_rent_fit.forecast(steps=1)

        model_return = ARIMA(station_data['AvailableReturnBikes'], order=(1,1,0))
        model_return_fit = model_return.fit()
        forecast_return = model_return_fit.forecast(steps=1)

        if not station_data.empty:
            if forecast_rent.iloc[0] <= 0.5:
                 stations_with_zero_rent.append(station_data['StationName'].iloc[0])

            if forecast_return.iloc[0] <= 0.5:
                  stations_with_zero_return.append(station_data['StationName'].iloc[0])

    return stations_with_zero_rent, stations_with_zero_return


In [ ]:
if __name__ == "__main__":
    while True:
        collect_data()
        data = pd.read_csv('國立中山大學幾何中心周圍1公里Youbike站點即時狀態.csv')

        if len(data) < 50:
            print("Waiting for more data...")
            time.sleep(600)
            continue

        stations_with_zero_rent, stations_with_zero_return = predict_zero_bikes_or_slots_with_arima(data)

        print("Stations predicted to have zero bikes available for rent in the next 10 mins:")
        print(stations_with_zero_rent)
        print("\nStations predicted to have zero slots available for return in the next 10 mins:")
        print(stations_with_zero_return)

        time.sleep(600)
